In [ ]:
from SocialNetwork.social_network_environment import *
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
from Advertising.learners.Greedy_algorithm import *
from experiment_utilities import *

seed = 6789 

arms = arms_creation(seed = 30291, number_of_arms= 20)

random.seed(seed)

exploration=4.2

deltas=[0.5,0.42,0.38,0.30,0.2,0.05]

social_network = social_network_environment()
social_network.init_from_json(json_path_network='Config/network.json', json_path_features='Config/features.json')


with open('Config/clairvoyants_sw.json', 'r') as file:
    clairvoyants = json.load(file)

with open('Config/bids.json', 'r') as file:
    data = json.load(file)
    bids = data.get('bids')


n_rounds = 20000
interval = 5000
window_size = 2000


In [2]:
from Advertising.learners.UCB_sw import *

vcg = VCG()
moving_average = []
moving_average.append(0)
learner = UCB_sw(arms = arms, hyperpar= exploration, window_size= window_size)

step = 0
for t in trange(n_rounds):
    if t % interval == 0:
        all_bids = bids[step]
        step +=1 

    arm_idx = learner.pull_arm()

    #print(arm_idx)
    arm = arms[arm_idx]
    all_bids.insert(0, arm)

    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)

    for j in range(0, len(best_allocation)):
        temp_allocation = []
        temp_allocation = best_allocation[j]
        for k in range(0, len(best_allocation[j])):
            if temp_allocation[k].ad_id == 1 and k != 0:
                temp_allocation[k].ad_id = None
                best_allocation[j] = temp_allocation
    
    
    payments = vcg.payments(ad_allocation_list, best_allocation, social_network)

    active_nodes, total_reward = activate_cascade(social_network=social_network, ad_allocation_list=best_allocation, slot_prominence=deltas)
    payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)
        
    reward = total_reward - payments_tot


    learner.update(pulled_arm_idx= arm_idx, reward = reward)
    moving_average.append((moving_average[-1] * t + learner.collected_rewards[-1]) / (t+1))

    reset_nodes(social_network=social_network)
    all_bids.pop(0)

pulls:24
pulls:168
pulls:13
pulls:44
pulls:15
pulls:12
pulls:33
pulls:29
pulls:27
pulls:21
pulls:19
pulls:241
pulls:35
pulls:5
pulls:17
pulls:24
pulls:13
pulls:10
pulls:17
pulls:17
pulls:24
pulls:169
pulls:13
pulls:44
pulls:15
pulls:12
pulls:33
pulls:29
pulls:27
pulls:21
pulls:19
pulls:241
pulls:35
pulls:5
pulls:17
pulls:24
pulls:13
pulls:10
pulls:17
pulls:17
pulls:24
pulls:170
pulls:13
pulls:44
pulls:15
pulls:12
pulls:33
pulls:29
pulls:27
pulls:21
pulls:19
pulls:241
pulls:35
pulls:5
pulls:17
pulls:24
pulls:13
pulls:10
pulls:17
pulls:17
pulls:24
pulls:171
pulls:13
pulls:44
pulls:15
pulls:12
pulls:33
pulls:29
pulls:27
pulls:21
pulls:19
pulls:241
pulls:35
pulls:5
pulls:17
pulls:24
pulls:13
pulls:10
pulls:17
pulls:17
pulls:24
pulls:172
pulls:13
pulls:44
pulls:15
pulls:12
pulls:33
pulls:29
pulls:27
pulls:21
pulls:19
pulls:241
pulls:35
pulls:5
pulls:17
pulls:24
pulls:13
pulls:10
pulls:17
pulls:17
pulls:24
pulls:173
pulls:13
pulls:44
pulls:15
pulls:12
pulls:33
pulls:29
pulls:27
pulls:21
pull

KeyboardInterrupt: 

In [ ]:
array = [i+1 for i in range(0, 20)]

print(learner.bounds)
print(learner.exp_values)
plt.figure(1)
plt.errorbar(array, learner.exp_values, yerr=learner.bounds, fmt='o')
plt.show()

In [ ]:

with open('Config/clairvoyants_sw.json', 'r') as file:
    clairvoyants = json.load(file)
clairvoyants_first = clairvoyants.get('clairvoyant_f_sw')
clairvoyants_all = clairvoyants.get('clairvoyant_a_sw')

clairvoyant_non_cum_only = []
for i in range(0, len(clairvoyants_first)):
    for j in range(0, interval):
        #clairvoyant_non_cum.append(expected_values_list_only[i][index])
        clairvoyant_non_cum_only.append(clairvoyants_first[i])

clairvoyant_cum_only = clairvoyant_non_cum_only.append(clairvoyant_non_cum_only[-1])
clairvoyant_cum_only = np.cumsum(clairvoyant_non_cum_only)


print("Graphic for TS with ads in all positions")
plt.figure(0)
plt.ylabel("Reward")
plt.xlabel("Rounds")
plt.plot(clairvoyant_cum_only, 'r')
plt.plot(np.cumsum(learner.collected_rewards), 'y')
plt.legend(["Clairvoyant", "UCB" ])



In [ ]:
print("Graphic for TS with ads in all positions")
plt.figure(0)
plt.ylabel("Reward")
plt.xlabel("Rounds")
plt.plot(clairvoyant_non_cum_only, 'r')
plt.plot(moving_average[100:], 'y')
plt.legend(["Clairvoyant", "UCB" ])

In [ ]:
plt.figure()
plt.plot(np.cumsum(learner.collected_rewards))


